In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re

In [2]:
# read in beerstyles csv
df = pd.read_csv('RonessaBeer.csv', encoding = "ISO-8859-1")

In [3]:
df.head()

,Unnamed: 0,names,style,url
0,5000,Amber,American Amber / Red Ale,https://www.beeradvocate.com/beer/profile/2781...
1,5001,Phobos Single Hop Red Ale,American Amber / Red Ale,https://www.beeradvocate.com/beer/profile/3305...
2,5002,Back Draft,American Amber / Red Ale,https://www.beeradvocate.com/beer/profile/2916...
3,5003,Briarpatch Amber Ale,American Amber / Red Ale,https://www.beeradvocate.com/beer/profile/2099...
4,5004,1982,American Amber / Red Ale,https://www.beeradvocate.com/beer/profile/1806...


In [4]:
urls = [row['url'] for index,row in df.iterrows()]

In [5]:
urls

['https://www.beeradvocate.com/beer/profile/27811/153228/',
 'https://www.beeradvocate.com/beer/profile/33058/182425/',
 'https://www.beeradvocate.com/beer/profile/29161/117925/',
 'https://www.beeradvocate.com/beer/profile/20998/52648/',
 'https://www.beeradvocate.com/beer/profile/18069/55739/',
 'https://www.beeradvocate.com/beer/profile/4399/25276/',
 'https://www.beeradvocate.com/beer/profile/1412/3787/',
 'https://www.beeradvocate.com/beer/profile/22243/63439/',
 'https://www.beeradvocate.com/beer/profile/14102/33491/',
 'https://www.beeradvocate.com/beer/profile/33518/109276/',
 'https://www.beeradvocate.com/beer/profile/1345/9183/',
 'https://www.beeradvocate.com/beer/profile/21473/53995/',
 'https://www.beeradvocate.com/beer/profile/33476/133360/',
 'https://www.beeradvocate.com/beer/profile/33657/114703/',
 'https://www.beeradvocate.com/beer/profile/28259/78359/',
 'https://www.beeradvocate.com/beer/profile/2085/6683/',
 'https://www.beeradvocate.com/beer/profile/38665/158752/

In [6]:
Score = [] #<span class="ba-ravg">2.36</span>
ScoreClass = []
RatingCount = []
Ranking = []
ReviewCount = []
pDev = [] #percent deviation
BrosScore = []
Brewer = []
Location = []
ABV = []
Availability = []
imageLink = []

beerCount = 1

# for index, row in df.iterrows():
#     print(row[2])
#     url = row[2]

# test 1 url first
# url = "https://www.beeradvocate.com/beer/profile/29/65/"
# urls = [row['url'] for row in df.iterrows()]

# use this to test 2 urls in a row
# urls = ['https://www.beeradvocate.com/beer/profile/29/65/','https://www.beeradvocate.com/beer/profile/285/39766/']
for url in urls:
    # go to each beer style page and get the top 50 beers (by count of ratings)
    # response from site
    try:
        print(beerCount)
        response = requests.get(url)

        # Create BeautifulSoup object; parse with 'html.parser'
        soup = BeautifulSoup(response.text, 'html.parser')

        beerScore = soup.find('span', class_='ba-ravg')
        if (beerScore.text is not None):
            Score.append(beerScore.text)
        else:
            Score.append('')

        scoreclass = soup.find('div', {'id':'score_box'}).find_all('b')
        if (scoreclass[1] is not None):
            ScoreClass.append(scoreclass[1].text)
        else:
            ScoreClass.append('')

        rating = soup.find('span', class_='ba-ratings')
        if (rating.text is not None):
            RatingCount.append(rating.text)
        else:
            RatingCount.append('')

        itemstats = soup.find('div', {'id': 'item_stats'}).find_all('dd')
        ranking = itemstats[0].text.replace("#", "")
        if (ranking is not None):
            Ranking.append(ranking)
        else:
            Ranking.append('')

        reviews = itemstats[1].text
        if (reviews is not None):
            ReviewCount.append(reviews)
        else:
            ReviewCount('')

        pdev = re.sub('\s','',itemstats[3].text).replace("%", "")
        if (pdev is not None):
            pDev.append(pdev)
        else:
            pDev.append('')

        brosdiv = soup.find('span', {'class': 'ba-bro_score'})
        #print(brosdiv)
        bros = brosdiv.text.strip()
        if (bros is not None):
            BrosScore.append(bros)
        else:
            BrosScore.append('')

        # get brewer and brewery from infobox    
        infobox = soup.find('div', {'id': 'info_box'}).find_all('a')

        brewer = infobox[0].find('b').text

        if (brewer is not None):
            Brewer.append(brewer)
        else:
            Brewer.append('')

        # what needs to be checked for empty here?
        # do i need to empty the list when i'm done using it for next time?
        location = []
        for ahref in infobox:
            if ('/place/' in ahref['href']):
                location.append(ahref.text)

                
        if (len(location) > 1):
            place = ', '.join(location)
        elif (len(location) == 1):
            place = location[0]
        else:
            place = ''
#         print(place)
        Location.append(place)

        # get ABV and availability from infoboxHeadings
        infoboxHeadings = soup.find('div', {'id': 'info_box'}).find_all('b')
        abv = re.sub('\s','',infoboxHeadings[5].next_sibling).replace("%", "")
        if (abv is not None):
            ABV.append(abv)
        else:
            ABV.append('')

        availability = re.sub('\s','',infoboxHeadings[6].next_sibling)
        if (availability is not None):
            Availability.append(availability)
        else:
            Availability.append('')

        beerimages = soup.find('div', {'id': 'main_pic_norm'}).find_all('img')
        beerlink = beerimages[0]['src']
        if (beerlink is not None):
            imageLink.append(beerlink)
        else:
            imageLink.append('')

        beerCount += 1
    except:
        print('error!', beerCount, url)
        beerCount += 1 #move to the next one

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


1861
1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060


3500
3501
3502
3503
3504
3505
3506
3507
3508
3509
3510
3511
3512
3513
3514
3515
3516
3517
3518
3519
3520
3521
3522
3523
3524
3525
3526
3527
3528
3529
3530
3531
3532
3533
3534
3535
3536
3537
3538
3539
3540
3541
3542
3543
3544
3545
3546
3547
3548
3549
3550
3551
3552
3553
3554
3555
3556
3557
3558
3559
3560
3561
3562
3563
3564
3565
3566
3567
3568
3569
3570
3571
3572
3573
3574
3575
3576
3577
3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
3622
3623
3624
3625
3626
3627
3628
3629
3630
3631
3632
3633
3634
3635
3636
3637
3638
3639
3640
3641
3642
3643
3644
3645
3646
3647
3648
3649
3650
3651
3652
3653
3654
3655
3656
3657
3658
3659
3660
3661
3662
3663
3664
3665
3666
3667
3668
3669
3670
3671
3672
3673
3674
3675
3676
3677
3678
3679
3680
3681
3682
3683
3684
3685
3686
3687
3688
3689
3690
3691
3692
3693
3694
3695
3696
3697
3698
3699


In [31]:
# scoreclass = soup.find('div', {'id':'score_box'}).find_all('b')
# sc = scoreclass[1].text
# print (sc)
# print(imageLink)

In [40]:
featuredf = pd.DataFrame({"Score":Score,"ScoreClass":ScoreClass,
                          "RatingCount":RatingCount,"Ranking":Ranking,"ReviewCount":ReviewCount,"pDev":pDev,
                         "BrosScore":BrosScore,"Brewer":Brewer,"Location":Location,"ABV":ABV,
                         "Availaility":Availability,"ImageLink":imageLink})

In [41]:
featuredf

,ABV,Availaility,Brewer,BrosScore,ImageLink,Location,Ranking,RatingCount,ReviewCount,Score,ScoreClass,pDev
0,5.00,Year-round,Anheuser-Busch,0,https://cdn.beeradvocate.com/im/beers/65.jpg,"Missouri, United States","45,313","6,308","1,713",2.36,Poor,33.47
1,4.74,Year-round,Pabst Brewing Company,3.65,https://cdn.beeradvocate.com/im/beers/1331.jpg,"California, United States","45,053","5,683","1,753",2.93,Poor,25.26
2,4.60,Year-round,Grupo Modelo S.A. de C.V.,1.89,https://cdn.beeradvocate.com/im/beers/232.jpg,Mexico,"45,315","5,071","1,261",2.34,Poor,31.62
3,4.60,Year-round,Miller Brewing Co.,3.53,https://cdn.beeradvocate.com/im/beers/580.jpg,"Wisconsin, United States","45,226","3,797","1,158",2.69,Poor,28.62
4,4.70,Year-round,Desnoes & Geddes Limited,3.39,https://cdn.beeradvocate.com/im/beers/689.jpg,Jamaica,"45,007","3,254",985,2.97,Poor,20.88
5,5.00,Year-round,Coors Brewing Company (Molson-Coors),3.05,https://cdn.beeradvocate.com/im/beers/1276.jpg,"Colorado, United States","45,266","2,967",741,2.59,Poor,28.57
6,4.60,Year-round,Latrobe Brewing Co.,2.86,https://cdn.beeradvocate.com/im/beers/567.jpg,"Missouri, United States","45,265","2,924",853,2.59,Poor,27.03
7,4.30,Year-round,"Cervecería Cuauhtémoc Moctezuma, S.A. de C.V.",2.65,https://cdn.beeradvocate.com/im/beers/1907.jpg,Mexico,"45,194","2,766",620,2.74,Poor,22.99
8,4.66,Year-round,Miller Brewing Co.,2.19,https://cdn.beeradvocate.com/im/beers/2280.jpg,"Wisconsin, United States","45,320","2,494",662,2.27,Poor,30.84
9,4.40,Year-round,Grupo Modelo S.A. de C.V.,3.4,https://cdn.beeradvocate.com/im/beers/1321.jpg,Mexico,"45,140","2,316",601,2.82,Poor,22.34


In [42]:
len(featuredf)

2400

In [7]:
# save to csv
featuredf.to_csv('RonessaFeatures.csv', index=False)

NameError: name 'featuredf' is not defined